In [ ]:
pip install git+https://github.com/billybraith17/billiespnet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/billybraith17/billiespnet to /tmp/pip-req-build-o1co0u06
  Running command git clone -q https://github.com/billybraith17/billiespnet /tmp/pip-req-build-o1co0u06
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 86 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 54.2 MB/s 
     |████████████████████████████████| 73 kB 1.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 212 kB 59.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 44.4 MB/s 
     |████████████████████████████████| 235 kB 57.0 MB/s 
     |████████████████████████████████| 3.1 MB 53.8 MB

In [ ]:
pip install torch espnet espnet_model_zoo parallel_wavegan==0.5.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.3 MB/s 
     |████████████████████████████████| 125 kB 12.0 MB/s 
     |████████████████████████████████| 120 kB 50.4 MB/s 
  Created wheel for parallel-wavegan: filename=parallel_wavegan-0.5.4-py3-none-any.whl size=71010 sha256=d5ccec633e05ebbd1bdd8e4d9e6edccd700b72a314b5a1b5c71f76fd16ab5e9f
  Stored in directory: /root/.cache/pip/wheels/5e/76/4d/8d938995675ca8cc9352214c9f23c57a505fe3bcd46ce3b138
Successfully built parallel-wavegan


In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
pip install ninja 2>> install.log
git clone https://github.com/SIDN-IAP/global-model-repr.git tutorial_code 2>> install.log
# git clone https://github.com/billybraith17/global-model-repr.git tutorial_code 2>> install.log

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
try: # set up path
    import google.colab, sys, torch
    sys.path.append('/content/tutorial_code')
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")
except:
    pass

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Running pytorch', torch.__version__, 'using', device.type)

Running pytorch 1.12.1+cu113 using cuda


In [ ]:
import torchvision
import torch.hub

# using the global-model-repr package
from netdissect import nethook, proggan

In [ ]:
# def from_pretrained2(
#     model_tag: Optional[str] = None,
#     vocoder_tag: Optional[str] = None,
#     **kwargs: Optional[Any],
# ):
"""Build Text2Speech instance from the pretrained model.
Args:
    model_tag (Optional[str]): Model tag of the pretrained models.
        Currently, the tags of espnet_model_zoo are supported.
    vocoder_tag (Optional[str]): Vocoder tag of the pretrained vocoders.
        Currently, the tags of parallel_wavegan are supported, which should
        start with the prefix "parallel_wavegan/".
Returns:
    Text2Speech: Text2Speech instance.
"""

from espnet2.bin.tts_inference import Text2Speech

model_tag = 'kan-bayashi/vctk_xvector_conformer_fastspeech2' #@param ["kan-bayashi/vctk_gst_tacotron2", "kan-bayashi/vctk_gst_transformer", "kan-bayashi/vctk_xvector_tacotron2", "kan-bayashi/vctk_xvector_transformer", "kan-bayashi/vctk_xvector_conformer_fastspeech2", "kan-bayashi/vctk_gst+xvector_tacotron2", "kan-bayashi/vctk_gst+xvector_transformer", "kan-bayashi/vctk_gst+xvector_conformer_fastspeech2", "kan-bayashi/vctk_multi_spk_vits", "kan-bayashi/vctk_full_band_multi_spk_vits", "kan-bayashi/libritts_xvector_transformer", "kan-bayashi/libritts_xvector_conformer_fastspeech2", "kan-bayashi/libritts_gst+xvector_transformer", "kan-bayashi/libritts_gst+xvector_conformer_fastspeech2", "kan-bayashi/libritts_xvector_vits"] {type:"string"}
vocoder_tag = "parallel_wavegan/vctk_hifigan.v1"  #@param ["none", "parallel_wavegan/vctk_parallel_wavegan.v1.long", "parallel_wavegan/vctk_multi_band_melgan.v2", "parallel_wavegan/vctk_style_melgan.v1", "parallel_wavegan/vctk_hifigan.v1", "parallel_wavegan/libritts_parallel_wavegan.v1.long", "parallel_wavegan/libritts_multi_band_melgan.v2", "parallel_wavegan/libritts_hifigan.v1", "parallel_wavegan/libritts_style_melgan.v1"] {type:"string"}
kwargs = {
    # Only for Tacotron 2 & Transformer
    "threshold" : 0.5,
    # Only for Tacotron 2
    "minlenratio" : 0.0,
    "maxlenratio" : 10.0,
    "use_att_constraint" : False,
    "backward_window" : 1,
    "forward_window" : 3,
    # Only for FastSpeech & FastSpeech2 & VITS
    "speed_control_alpha" : 1.0,
    # Only for VITS
    "noise_scale" : 0.333,
    "noise_scale_dur" : 0.333,
}

# print(kwargs["noise_scale"])
# model_tag: Optional[str] = None
# vocoder_tag: Optional[str] = None
# **kwargs: Optional[Any]


if model_tag is not None:
    try:
        from espnet_model_zoo.downloader import ModelDownloader

    except ImportError:
        logging.error(
            "`espnet_model_zoo` is not installed. "
            "Please install via `pip install -U espnet_model_zoo`."
        )
        raise
    d = ModelDownloader()
    kwargs.update(**d.download_and_unpack(model_tag))

if vocoder_tag is not None:
    if vocoder_tag.startswith("parallel_wavegan/"):
        try:
            from parallel_wavegan.utils import download_pretrained_model

        except ImportError:
            logging.error(
                "`parallel_wavegan` is not installed. "
                "Please install via `pip install -U parallel_wavegan`."
            )
            raise

        from parallel_wavegan import __version__
        from packaging.version import parse as V
        from pathlib import Path

        # NOTE(kan-bayashi): Filelock download is supported from 0.5.2
        assert V(__version__) > V("0.5.1"), (
            "Please install the latest parallel_wavegan "
            "via `pip install -U parallel_wavegan`."
        )
        vocoder_tag = vocoder_tag.replace("parallel_wavegan/", "")
        vocoder_file = download_pretrained_model(vocoder_tag)
        vocoder_config = Path(vocoder_file).parent / "config.yml"
        kwargs.update(vocoder_config=vocoder_config, vocoder_file=vocoder_file)

    else:
        raise ValueError(f"{vocoder_tag} is unsupported format.")

# __init__
text2speech = Text2Speech(**kwargs)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
https://zenodo.org/record/4394602/files/tts_train_xvector_conformer_fastspeech2_transformer_teacher_raw_phn_tacotron_g2p_en_no_space_train.loss.ave.zip?download=1: 100%|██████████| 270M/270M [00:32<00:00, 8.69MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oVOC4Vf0DYLdDp4r7GChfgj7Xh5xd0ex
To: /root/.cache/parallel_wavegan/vctk_hifigan.v1.tar.gz
100%|██████████| 916M/916M [00:17<00:00, 52.1MB/s]


In [ ]:
# Wrap the vocoder
from netdissect import nethook
layer = 'vocoder.blocks.7.convs1.2.1'    #@param ["vocoder.last_conv_layers.0", "vocoder.last_conv_layers.1", "vocoder.last_conv_layers.2", "vocoder.last_conv_layers.3", "vocoder.conv_layers.6.conv", "vocoder.conv_layers.6.conv1x1_aux", "vocoder.conv_layers.6.conv1x1_out", "vocoder.conv_layers.6.conv1x1_skip", "vocoder.first_conv", "vocoder.upsample_net.conv_in", "vocoder.upsample_net.upsample.up_layers.6", "vocoder.upsample_net.upsample.up_layers.7", "vocoder.melgan.0", "vocoder.melgan.19.stack.4", "vocoder.melgan.23", "vocoder.pqmf.pad_fn", "vocoder.blocks.7.upsample", "vocoder.output_conv.1", "vocoder.blocks.11.convs2.0.1", "vocoder.output_conv.2", "vocoder.blocks.0.convs1.1.1", "vocoder.blocks.1.convs1.1.1", "vocoder.blocks.4.convs1.1.1", "vocoder.blocks.7.convs1.1.1", "vocoder.blocks.7.convs1.2.1", "vocoder.blocks.8.convs1.2.1", "vocoder.blocks.9.convs1.1.1", "vocoder.blocks.9.convs2.1.1",  "vocoder.blocks.10.convs1.1.1",  "vocoder.blocks.10.convs2.1.1", "vocoder.blocks.11.convs2.1.1", "vocoder.blocks.0.convs1.1.0", "vocoder.blocks.1.convs1.1.0", "vocoder.blocks.4.convs1.1.0", "vocoder.blocks.7.convs1.1.0", "vocoder.blocks.7.convs1.2.0", "vocoder.blocks.8.convs1.2.0", "vocoder.blocks.9.convs1.1.0", "vocoder.blocks.9.convs2.1.0",  "vocoder.blocks.10.convs1.1.0",  "vocoder.blocks.10.convs2.1.0", "vocoder.blocks.11.convs2.1.0"] {type:"string"}


# Don't re-wrap it, if it's already wrapped (e.g., if you press enter twice)
if not isinstance(text2speech.vocoder, nethook.InstrumentedModel):
    text2speech.vocoder = nethook.InstrumentedModel(text2speech.vocoder)
text2speech.vocoder.retain_layer(layer)

In [ ]:
text2speech.vocoder

# text2speech.vocoder.retain_layer('vocoder')

InstrumentedModel(
  (model): ParallelWaveGANPretrainedVocoder(
    (vocoder): HiFiGANGenerator(
      (input_conv): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
      (upsamples): ModuleList(
        (0): Sequential(
          (0): LeakyReLU(negative_slope=0.1)
          (1): ConvTranspose1d(512, 256, kernel_size=(10,), stride=(5,), padding=(3,), output_padding=(1,))
        )
        (1): Sequential(
          (0): LeakyReLU(negative_slope=0.1)
          (1): ConvTranspose1d(256, 128, kernel_size=(10,), stride=(5,), padding=(3,), output_padding=(1,))
        )
        (2): Sequential(
          (0): LeakyReLU(negative_slope=0.1)
          (1): ConvTranspose1d(128, 64, kernel_size=(8,), stride=(4,), padding=(2,))
        )
        (3): Sequential(
          (0): LeakyReLU(negative_slope=0.1)
          (1): ConvTranspose1d(64, 32, kernel_size=(6,), stride=(3,), padding=(2,), output_padding=(1,))
        )
      )
      (blocks): ModuleList(
        (0): HiFiGANResidualB

In [ ]:
import glob
import os
import numpy as np
import kaldiio

# Get model directory path
from espnet_model_zoo.downloader import ModelDownloader
d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(model_tag)["train_config"])

# X-vector selection
# We only need the x-vector part at that's what this model uses. The ones with
spembs = None
if text2speech.use_spembs:
  xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
  xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
  spks = list(xvectors.keys())

In [ ]:
from netdissect import tally
x = "Please call Stella. Ask her too."

# To collect stats, define a function that returns 2d [samples, units]
def compute_samples(index):
    # print(index)
    spk = spks[int(index[0])]
    spembs = xvectors[spk]
    wav = text2speech(x, spembs=spembs)["wav"]
    acts = text2speech.vocoder.retained_layer(layer)
    acts = torch.flatten(acts, start_dim=1)
    acts_stack = acts
    # print(acts.shape)
    # print(acts)
    # acts_stack = acts[0]

    for i in index[1:]:
      spk = spks[int(i)]
      spembs = xvectors[spk]
      wav = text2speech(x, spembs=spembs)["wav"]
      # acts = model.retained_layer('layer4') # get the activations, and flatten
      acts = text2speech.vocoder.retained_layer(layer)
      # print(acts[0].shape)
      # print(acts)
      acts = torch.flatten(acts, start_dim=1)

      acts_stack = torch.cat((acts_stack, acts), 0)
      # print(acts_stack.shape)
    return acts_stack


from torch.utils.data import TensorDataset
# Define our batch of inputs to the model
# zds = TensorDataset(torch.linspace(0, 49, 50))
# this also works
zds = torch.linspace(0, 107, 108)

# Then tally_quantile will run your function over the whole dataset to collect quantile stats
# rq = tally.tally_quantile(compute_samples, zds, batch_size=10)

# Print out the median value for the first 20 channels
# print(rq.quantiles(0.9)[:20])
# print(rq.mean())

In [ ]:
# print(rq.quantiles(1)[:20])

In [ ]:
# Then tally_quantile will run your function over the whole dataset to collect quantile stats
rq = tally.tally_quantile(compute_samples, zds, batch_size=10)

# Print out the median value for the first 20 channels
print(rq.quantiles(0.99).shape)

  0%|          | 0/11 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/espnet2/torch_utils/device_funcs.py:29: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:172.)
  return to_device(torch.from_numpy(data), device, dtype, non_blocking, copy)


torch.Size([640000])


In [ ]:
topk = tally.tally_topk(compute_samples, zds)
topk.result()[1].shape

torch.Size([640000, 100])

In [ ]:
text2speech.vocoder.retained_layer(layer).shape

torch.Size([1, 64, 10000])

In [ ]:
topk.result()[1]

tensor([[ 86,  60,  24,  ...,  54,   1,  84],
        [ 42,  24,   9,  ...,  39,  53,  62],
        [ 84,  42,  74,  ...,  41,   5,  36],
        ...,
        [  2,  98, 105,  ...,  49,  15,  60],
        [  2,  91,  33,  ...,  96,  56,  53],
        [ 33,   2,  79,  ...,  98,  19,  22]])

In [ ]:
topk.result()[0]

tensor([[ 0.1696,  0.0834,  0.0642,  ..., -0.1190, -0.1202, -0.1289],
        [ 0.0842,  0.0445,  0.0374,  ..., -0.1900, -0.2051, -0.2116],
        [ 0.1456,  0.1282,  0.1007,  ..., -0.1156, -0.1160, -0.1197],
        ...,
        [ 1.0330,  0.4251,  0.3361,  ..., -0.7525, -0.7858, -0.7922],
        [ 1.0480,  0.3974,  0.3046,  ..., -0.8288, -0.9122, -0.9425],
        [ 0.6343,  0.6240,  0.4065,  ..., -0.8183, -0.8272, -0.8362]])

In [ ]:
southern_units = {0, 3, 4, 6, 7, 14, 30, 31, 41}
s_african_units = {82, 86, 93, 99}
indian = {21, 24, 107}
scottish = {9, 11, 15, 19, 20, 22, 25, 28, 33, 35, 36, 37, 38, 44, 45, 48, 54, 57, 58}
irish = {18, 39, 56, 61, 65, 67, 81, 95, 105}
dublin = {18, 61, 65, 95}
n_irish = {12, 34, 62, 63, 73, 100}
belfast = {12, 34, 62, 63, 73}
cali = {64, 68, 69, 85}
yorkshire = {40, 43, 60}
NE = {42, 50, 55, 59}
canada = {71, 72, 76, 80, 83, 84, 97, 104}

female = {0, 3,4,5,6, 8, 9, 10, 12, 13, 14, 17, 21, 22, 23, 26, 30, 34, 35, 37, 38, 39, 40, 41, 42, 49, 50, 53, 55, 56, 61, 63, 64, 65, 66, 68, 69, 70, 72, 74, 75, 76, 77, 78, 80, 81,82, 84, 85, 86, 88, 89, 90, 92,93,94,95,96,97, 100, 102,103}
male = {1, 2, 7, 11, 15, 16, 18, 19, 20, 24, 25, 27, 28, 29, 31, 32, 33, 36, 43, 44, 45, 46, 47, 48, 51, 52, 54, 57, 58, 59, 60, 62, 67, 71, 73, 79, 83, 87, 91, 98, 99, 101, 104, 105, 106, 107}

In [ ]:
# selecting the first 5 indices from the first row / unit
top = topk.result()[1][0][0:5]
print(top)
# casting from tensor to set
top = set(top.tolist())

# calculating IoU
num = top.intersection(southern_units)
print(num)
den = top.union(southern_units)
print(den)
IoU = len(num) / len(den)
print(IoU)

tensor([86, 60, 24, 15,  0])
{0}
{0, 3, 4, 6, 7, 41, 14, 15, 86, 24, 60, 30, 31}
0.07692307692307693


In [ ]:
# Quick method
# iterating st we can calculate the IoU for all units in the layer
IoU_list = []
chosen_property = male
result = topk.result()[1]
result = result.tolist()
# len(result)

for i in range(0, topk.result()[1].shape[0]):
  top = result[i][0:5]
  top = set(top)
  num = top.intersection(chosen_property)
  den = top.union(chosen_property)
  IoU_list.append(len(num) / len(den))
  # print(i)

# top = set(top.tolist())
# for i in range(0, topk.result()[1].shape[0]):
#   num = top.intersection(southern_units)
#   den = top.union(southern_units)
#   IoU_list.append(len(num) / len(den))
#   print(i)

In [ ]:
# same as above but to zero units
IoU_list_z = []
property_z = female

for i in range(0, topk.result()[1].shape[0]):
  top = result[i][0:5]
  top = set(top)
  num = top.intersection(property_z)
  den = top.union(property_z)
  IoU_list_z.append(len(num) / len(den))
  # print(i)

In [ ]:
len(IoU_list)

640000

In [ ]:
# IoU_list

In [ ]:
# setting the number of units we want to change the actication to the top 1% quantile for and printing their indices
n_units = 50000
IoU_list = torch.tensor(IoU_list)
top_IoU = torch.topk(IoU_list, n_units)
# print(layer, '\n', top_IoU, '\n')
rand_IoU = torch.randint(0, len(IoU_list), (n_units,))

In [ ]:
# setting the number of units we want to change the activations of to 0 and printing their indices
n_units_z = 5000
IoU_list_z = torch.tensor(IoU_list_z)
top_IoU_z = torch.topk(IoU_list_z, n_units_z)
print(layer, '\n', top_IoU_z, '\n')
rand_IoU_z = torch.randint(0, len(IoU_list), (n_units_z,))


vocoder.blocks.7.convs1.2.1 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806,  ..., 0.0806, 0.0806, 0.0806]),
indices=tensor([ 7037,  6672,  6991,  ..., 22582, 22391,  8279])) 



In [ ]:
# top_IoU[1][1]

In [ ]:
# defining functions to get the unit's index from the number layer and its index from topk
def index_map(layer1, idx):
    div = text2speech.vocoder.retained_layer(layer1).shape[2]
    first = idx//div
    second = idx%div
    return [0, first, second]


# index_map_list(layer, top_IoU[1].numpy())

In [ ]:
# function to set units to the top 1% quantile
def change_units(data, model):
    for i in range(0, n_units_z):
#     if want to only change the top IoU nodes
      temp = index_map(layer, top_IoU_z[1][i])
      # if want to change the same number of random nodes
      # temp = index_map(layer, rand_IoU_z[i])
      data[0, temp[1], temp[2]] = 0

    quant = rq.quantiles(0.99)
    for i in range(0, n_units):
#     if want to only change the top IoU nodes
      temp = index_map(layer, top_IoU[1][i])
      # if want to change the same number of random nodes
      # temp = index_map(layer, rand_IoU[i])
      data[0, temp[1], temp[2]] = quant[top_IoU[1][i]]

    return data

# setting units to top 1%
text2speech.vocoder.edit_layer(layer, rule=change_units)


# # function to set units to 0
# def zero_units(data, model):
#     for i in range(0, n_units_z):
#       temp = index_map(layer, top_IoU_z[1][i])
#       data[0, temp[1], temp[2]] = 0

#     return data

# setting units to top 1%
# text2speech.vocoder.edit_layer(layer, rule=zero_units)

In [ ]:
spk = spks[10]
spembs = xvectors[spk]
print(f"selected spk: {spk}")

selected spk: p236


In [ ]:
import soundfile as sf
import time
# x = "Please call Stella. Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station."
x = "Please call Stella.  Ask her too."
# __call__
wav = text2speech(x, spembs=spembs)["wav"]
# print(text2speech(x, spembs=spembs)["feat_gen"].shape)
# print(text2speech(x, spembs=spembs)["feat_gen_denorm"].shape)

sf.write("out.wav", wav.numpy(), 24000, "PCM_16")

# Display the audio file below
from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=24000))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  after removing the cwd from sys.path.


In [ ]:
text2speech.vocoder.remove_edits()

In [ ]:
x = "Please call Stella.  Ask her too."
# __call__
wav = text2speech(x, spembs=spembs)["wav"]
# print(text2speech(x, spembs=spembs)["feat_gen"].shape)
# print(text2speech(x, spembs=spembs)["feat_gen_denorm"].shape)

sf.write("out.wav", wav.numpy(), 24000, "PCM_16")

# Display the audio file below
from IPython.display import display, Audio
display(Audio(wav.view(-1).cpu().numpy(), rate=24000))

In [ ]:
# testing the units had chage to e.g. 1 when we edited them using our new function
text2speech.vocoder.retained_layer(layer)[0][30][500:520]

tensor([1.2840, 1.4238, 1.3848, 1.6113, 1.3814, 1.5934, 1.3483, 1.4226, 1.4910,
        1.5604, 1.4348, 1.5434, 1.4218, 1.4007, 1.4654, 1.6396, 1.3522, 1.4603,
        1.4629, 1.5300])

In [ ]:
text2speech.vocoder.retained_layer(layer).shape

torch.Size([1, 64, 10000])

In [ ]:
import librosa

# librosa.feature.mfcc(y=y, sr=sr)

